In [1]:
# nse scratchpad initiation
from helper import *
from nse_func import *

# Do assignments
a = assign_var('nse')
for v in a:
    exec(v)

from ib_insync import *
util.startLoop()

In [4]:
ib =  get_connected('nse', 'live')

In [13]:
# get the remaining quantities
dfrq = remqty_nse(ib)

dft = pd.read_pickle(fspath+'targets.pickle')

dft = dft.assign(remqty=dft.symbol.map(dfrq.remqty.to_dict()))   # remaining quantities
dft = dft[dft.remqty > 0].reset_index(drop=True) # remove blacklisted (remqty <= 0)

In [41]:
mask = (dft.right == 'P') & (dft.undPrice-dft.strike < dft.Fall)
pd.merge(dft[mask][['symbol', 'strike', 'right', 'dte', 'undPrice', 'optPrice', 'expPrice', 'Fall']], pd.DataFrame((dft[mask].undPrice-dft[mask].strike)), on=dft[mask].index)

,key_0,symbol,strike,right,dte,undPrice,optPrice,expPrice,Fall,0
0,0,ACC,1240.0,P,47,1596.35,0.20,28.25,359.35,356.35
1,2,AJANTPHAR,880.0,P,47,1064.40,6.90,118.70,533.70,184.40
2,3,AJANTPHAR,860.0,P,47,1064.40,4.95,118.70,533.70,204.40
3,4,AJANTPHAR,840.0,P,47,1064.40,3.45,118.70,533.70,224.40
4,5,ALBK,33.0,P,19,44.50,0.10,0.35,25.20,11.50
5,6,ALBK,32.0,P,19,44.50,0.05,0.35,25.20,12.50
6,7,AMARAJABA,550.0,P,47,635.55,1.25,11.20,204.40,85.55
7,8,AMARAJABA,540.0,P,47,635.55,0.80,11.20,204.40,95.55
8,9,AMARAJABA,560.0,P,19,635.55,0.25,4.55,171.45,75.55
9,10,AMBUJACEM,185.0,P,47,213.95,0.60,3.90,57.20,28.95
